In [1]:
from classes.ClockDataset import *
from classes.CommonSenseError import *
from classes.Models import *
from utilities import *
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
weights_path = 'model_weights/periodic_labels_2'
batch_size=64

<h4>In this notebook we evaluate the pretrained periodic labelled approach on the whole dataset in order to calculate the 'common sense error' (true distance between predictions and targets)</h4>

In [2]:
# load model and loss function
model = NN_regression(  input_channels=1,
                        h=150,w=150,
                        n_outputs=4).to(device)
model.load_state_dict(torch.load(weights_path))
loss = nn.MSELoss()

In [3]:
# load data and create dataset
data, labels = load_data()
labels = transform_labels(labels)
clock_dataset = ClockDataset(data, labels)
data_loader = DataLoader(clock_dataset, batch_size=batch_size)

In [5]:
# make predictions
approach = "periodic_labels"
predictions = predict(data_loader, model, loss, device, approach)
true_preds = denormalize_time(predictions)

# define common sense error
cse = CommonSenseError()
cse_error = cse(torch.FloatTensor(true_preds),torch.FloatTensor(labels[:,:2]))

print(f"Mean error in minutes of predictions: {cse_error}")

Avg evaluation loss: 0.003286 

Mean error in minutes of predictions: 7.378833293914795
